## Lies `cars.csv` in einen DataFrame

In [2]:
import pandas as pd

cars = pd.read_csv('cars.csv')
cars.describe()

,price,yearOfRegistration,powerPS,kilometer
count,250.000000,250.000000,250.000000,250.00000
mean,8553.384000,2006.448000,118.068000,84520.00000
std,7832.009205,7.103883,49.183202,38265.21274
min,0.000000,1974.000000,41.000000,5000.00000
25%,2820.000000,2002.000000,75.250000,60000.00000
50%,5900.000000,2008.000000,109.000000,90000.00000
75%,12000.000000,2011.000000,150.000000,125000.00000
max,44000.000000,2018.000000,280.000000,125000.00000


## Data Cleaning

* entferne die Features `model` und `name`
* entferne Observations mit `NaN`-Einträgen
* entferne Observations, deren `fuelType` nicht `benzin` oder `diesel` ist
* Führe ein One-Hot-Encoding für `fuelType` durch

In [3]:
cars = cars.drop(['model', 'name'], axis=1)
cars = cars.dropna()
cars = cars[cars["fuelType"].isin(['benzin', 'diesel'])]
fuelType = pd.get_dummies(cars['fuelType'], drop_first=True)
cars = cars.drop(['fuelType'], axis=1)
cars = pd.concat([cars, fuelType], axis=1)
cars.head()


,price,yearOfRegistration,powerPS,kilometer,diesel
0,1450,1997,75,90000,0
1,13100,2005,280,5000,0
2,4500,2008,87,90000,0
3,6000,2009,177,125000,1
4,3990,1999,118,90000,0


## Training

* Verwende 20% Testdaten und `random_state=42`
* predicte `price`
* Verwende eine LinearRegression und ermittle, welches Feature einen negativen Einfluss auf den Preis hat
* Verwende einen RandomForestRegressor und ermittle das wichtigste Feature

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
X = cars.drop(['price'], axis=1)
y = cars['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
priceLinReg = linear_model.LinearRegression()
priceLinReg.fit(X_train, y_train)
pd.DataFrame([priceLinReg.coef_, X.columns])

,0,1,2,3
0,131.662959,84.224383,-0.079952,4997.553764
1,yearOfRegistration,powerPS,kilometer,diesel


In [6]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

priceForestReg = RandomForestRegressor()
priceForestReg.fit(X_train, y_train)
pd.DataFrame([priceForestReg.feature_importances_,X.columns])

,0,1,2,3
0,0.304028,0.44865,0.212051,0.035271
1,yearOfRegistration,powerPS,kilometer,diesel


## Evaluierung

* Ermittle einen durschnittlichen Fehler für beide Modelle

In [7]:
from sklearn import metrics
print(metrics.mean_absolute_error(y_test, priceLinReg.predict(X_test)))
print(metrics.mean_absolute_error(y_test, priceForestReg.predict(X_test)))

2681.997339586527
2489.3779047619046
